## Black-Scholes PDE
The `Black-Scholes partial differential equation (PDE)` for option $V$ is:
$$
\frac{\partial V}{\partial t} + r S_t \frac{\partial V}{\partial S} +  \frac{1}{2} \sigma^2 S_t^2 \frac{\partial^2 V}{\partial S^2} - r V_t = 0 $$
Writing $\Delta_t=\frac{\partial V}{\partial S_t}$ and $\Gamma_t=\frac{\partial^2 V}{\partial S_t^2}$, we get
$$\frac{\partial V}{\partial t} + r S_t \Delta_t + \frac{1}{2} \sigma^2 S_t^2 \Gamma_t - r V_t = 0 
$$

## The Greeks
We can measure the sensitivity of the price to market conditions using partial derivatives of $V$.
These partial derivatives are collectively called `The Greeks.` These can be seen as some kind of Risk-measure since they measure sensitivity to changes in the market. As the partial derivatives are linear, we can add up the Greeks of the components to compute the Greeks of the portfolio.
If a trader writes (i.e. sells) a call option and replicates it using the delta-hedging strategy, then the delta of their portfolio is 0 at all times. This is because their portfolio consists of $-1$ units of the option and $\Delta$ units of the stock. So the total $\Delta$ is $-1 \cdot  \Delta + 1 \cdot \Delta=0$, since the $\Delta$ of the stock is 1 ($\frac{\partial S}{\partial S}=1$). So the $\Delta$ hedging strategy can be viewed
as the strategy of ensuring that this particular risk-measure is always 0. In practice, derivatives traders manage their portfolios to ensure that most of the Greek's are low as, at least heuristically, one would then expect that the portfolio is hedged against risks such as a change in interest rate or volatility.
$$
\begin{split}
\Delta&:=\frac{\partial V}{\partial S} \\ 
\Gamma&:=\frac{\partial^2 V}{\partial S^2} \\
\rho&:=\frac{\partial V}{\partial r} \\
\theta&:=\frac{\partial V}{\partial t} \\
\text{Vega}&:=\frac{\partial V}{\partial \sigma}
\end{split}
$$

### Calculating formula for $\Delta$ using `sympy`
We begin by creating the symbols we need and then compute $V$ according to the Black-Scholes formula.
We introduce a variable $\tau=T-t$. We also indicate that all our variables will be positive as this will allow more simplification rules to be applied later. Since $N$ is a symbolic function rather than a variable, we must create it using `Function` command instead of using `symbols`.

In [4]:
import sympy as sp

S, tau, K, r, sigma, D1, D2 = sp.symbols('S tau K r sigma D1 D2',positive=True)
N = sp.Function('N')
d1 = 1/(sigma*sp.sqrt(tau))*(sp.log(S/K)+(r+sigma**2/2)*tau)
d2 = d1 - sigma * sp.sqrt(tau)
V = S*N(d1) -K*sp.exp(-r*tau)*N(d2)
display(V)
Delta = sp.diff(V,S)
Delta

-K*N(-sigma*sqrt(tau) + (tau*(r + sigma**2/2) + log(S/K))/(sigma*sqrt(tau)))*exp(-r*tau) + S*N((tau*(r + sigma**2/2) + log(S/K))/(sigma*sqrt(tau)))

-K*exp(-r*tau)*Subs(Derivative(N(_xi_1), _xi_1), _xi_1, -sigma*sqrt(tau) + (tau*(r + sigma**2/2) + log(S/K))/(sigma*sqrt(tau)))/(S*sigma*sqrt(tau)) + N((tau*(r + sigma**2/2) + log(S/K))/(sigma*sqrt(tau))) + Subs(Derivative(N(_xi_1), _xi_1), _xi_1, (tau*(r + sigma**2/2) + log(S/K))/(sigma*sqrt(tau)))/(sigma*sqrt(tau))

Notice that we obtain an expression for $\Delta$ in terms of the function $N$ and its partial derivative. We know the partial derivative of $N$ satisfies
$$
\frac{d N(x)}{dx}=\frac{1}{\sqrt{2 \pi}}\exp{-x^2/2}
$$
so we can simplify these derivative terms. What we want to do is take our expression for $\Delta$ and replace all partial derivatives of the form
$$
\frac{d N(*_x)}{d *_x}
$$
with
$$
\frac{1}{\sqrt{2 \pi}}e^{-\frac{*_x^2}{2}}.
$$
In these last two expressions $*_x$ could be any variable at all, but it must be the same expression each time it occurs. The label $x$ on
the pattern indicates that all the $*_x$ expressions must be identical. You can make changes to a `sympy` expression using the `replace` function. 

In [10]:
Delta = Delta.replace(sp.Derivative(N(sp.Wild('x')),sp.Wild('x')), sp.exp(-sp.Wild('x')**2/2)/(sp.sqrt(2*sp.pi)))
Delta = Delta.simplify()
Delta 

N((2*r*tau + sigma**2*tau + log(S**2/K**2))/(2*sigma*sqrt(tau)))

This can be written very neatly as
$$\Delta = N(d_1).$$
We check this on the next line to make sure by simlifying $\Delta - N(d_1)$. We expect zero.

In [11]:
(N(d1)-Delta).simplify()

0

- Lets use `sympy` to show that $V$ satisfies the Black-Scholes PDE
$$
\frac{\partial V}{\partial t} + r S \frac{\partial V}{\partial S } + \frac{1}{2} \sigma^2 S^2 \frac{\partial^2 V}{\partial S^2 } - r V = 0.
$$

In [12]:
Gamma = sp.diff( Delta, S )
Gamma = Gamma.replace(sp.Derivative(N(sp.Wild('x')),sp.Wild('x')), sp.exp(-sp.Wild('x')**2/2)/(sp.sqrt(2*sp.pi)))
Gamma = Gamma.simplify()
Gamma

sqrt(2)*exp(-(2*r*tau + sigma**2*tau + log(S**2/K**2))**2/(8*sigma**2*tau))/(2*sqrt(pi)*S*sigma*sqrt(tau))

In [14]:
theta = -sp.diff(V, tau)
theta = theta.replace(sp.Derivative(N(sp.Wild('x')),sp.Wild('x')), sp.exp(-sp.Wild('x')**2/2)/(sp.sqrt(2*sp.pi)))
theta = theta.simplify()
theta

-(4*K*r*sigma*tau**(9/2)*N((2*r*tau - sigma**2*tau + log(S**2/K**2))/(2*sigma*sqrt(tau))) + sqrt(2)*K*tau**3*(2*sigma**2*tau - tau*(2*r + sigma**2) + log(S**2/K**2))*exp(-(-2*sigma**2*tau + tau*(2*r + sigma**2) + log(S**2/K**2))**2/(8*sigma**2*tau))/(2*sqrt(pi)) + sqrt(2)*S*tau**3*(tau*(2*r + sigma**2) - log(S**2/K**2))*exp(r*tau)*exp(-(tau*(2*r + sigma**2) + log(S**2/K**2))**2/(8*sigma**2*tau))/(2*sqrt(pi)))*exp(-r*tau)/(4*sigma*tau**(9/2))

In [16]:
black_scholes_pde = theta + r*S*Delta + sigma**2*S**2*Gamma/2 - r*V
black_scholes_pde = black_scholes_pde.simplify()
assert black_scholes_pde == 0
black_scholes_pde

0